Простой пример: Сохранение состояния Редактора (одна строка текста)
Имитируем простой редактор, который хранит одну строку текста, и мы хотим иметь возможность сохранить и восстановить эту строку.

In [1]:
import time

# 1. Memento (Снимок)
class EditorMemento:
    """
    Снимок: хранит состояние Originator'а.
    В Python часто делают атрибуты 'приватными' по соглашению (_).
    """
    def __init__(self, content: str):
        self._saved_content = content
        self._timestamp = time.time() # Доп. информация (не обязательно)
        print(f"Memento: Created with content '{self._saved_content}'")

    # Метод, доступный только Originator'у (по соглашению)
    def _get_saved_content(self) -> str:
        return self._saved_content

    # Метод для Caretaker (может быть полезен)
    def get_info(self) -> str:
        return f"Memento saved at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(self._timestamp))}"


# 2. Originator (Создатель)
class TextEditor:
    """Объект, чьё состояние мы сохраняем (Редактор)."""
    def __init__(self):
        self._content = ""
        print("TextEditor: Initialized.")

    def type(self, text: str):
        print(f"Editor: Appending text '{text}'")
        self._content += text
        print(f"Editor: Current content: '{self._content}'")

    def get_content(self) -> str:
        return self._content

    def save(self) -> EditorMemento:
        """Создает снимок текущего состояния."""
        print("Editor: Saving current state...")
        return EditorMemento(self._content)

    def restore(self, memento: EditorMemento):
        """Восстанавливает состояние из снимка."""
        # Originator имеет доступ к 'приватным' методам Memento
        self._content = memento._get_saved_content()
        print(f"Editor: State restored from {memento.get_info()}.")
        print(f"Editor: Current content: '{self._content}'")


# 3. Caretaker (Опекун)
class History:
    """Опекун: хранит снимки (в данном случае, только один)."""
    def __init__(self):
        self._last_memento: EditorMemento | None = None
        print("History (Caretaker): Initialized.")

    def save_state(self, editor: TextEditor):
        """Запрашивает и сохраняет снимок."""
        print("History: Requesting save...")
        self._last_memento = editor.save()
        print(f"History: Saved state. {self._last_memento.get_info()}")

    def restore_state(self, editor: TextEditor):
        """Передает сохраненный снимок для восстановления."""
        if not self._last_memento:
            print("History: No state saved to restore.")
            return
        print("History: Requesting restore...")
        editor.restore(self._last_memento)
        print("History: Restore request sent.")


# 4. Client Code (Клиентский Код)
if __name__ == "__main__":
    editor = TextEditor()
    history = History()

    print("\n--- Typing and Saving ---")
    editor.type("This is the first sentence. ")
    history.save_state(editor) # Сохраняем первое состояние

    print("\n--- Typing more ---")
    editor.type("And this is the second.")
    print(f"Current content before restore: '{editor.get_content()}'")

    print("\n--- Restoring previous state ---")
    history.restore_state(editor) # Восстанавливаем предыдущее состояние
    print(f"Current content after restore: '{editor.get_content()}'")

# Вывод:
# TextEditor: Initialized.
# History (Caretaker): Initialized.
#
# --- Typing and Saving ---
# Editor: Appending text 'This is the first sentence. '
# Editor: Current content: 'This is the first sentence. '
# History: Requesting save...
# Editor: Saving current state...
# Memento: Created with content 'This is the first sentence. '
# History: Saved state. Memento saved at <Timestamp>
#
# --- Typing more ---
# Editor: Appending text 'And this is the second.'
# Editor: Current content: 'This is the first sentence. And this is the second.'
# Current content before restore: 'This is the first sentence. And this is the second.'
#
# --- Restoring previous state ---
# History: Requesting restore...
# Editor: State restored from Memento saved at <Timestamp>.
# Editor: Current content: 'This is the first sentence. '
# History: Restore request sent.
# Current content after restore: 'This is the first sentence. '

TextEditor: Initialized.
History (Caretaker): Initialized.

--- Typing and Saving ---
Editor: Appending text 'This is the first sentence. '
Editor: Current content: 'This is the first sentence. '
History: Requesting save...
Editor: Saving current state...
Memento: Created with content 'This is the first sentence. '
History: Saved state. Memento saved at 2025-04-21 11:42:26

--- Typing more ---
Editor: Appending text 'And this is the second.'
Editor: Current content: 'This is the first sentence. And this is the second.'
Current content before restore: 'This is the first sentence. And this is the second.'

--- Restoring previous state ---
History: Requesting restore...
Editor: State restored from Memento saved at 2025-04-21 11:42:26.
Editor: Current content: 'This is the first sentence. '
History: Restore request sent.
Current content after restore: 'This is the first sentence. '


Сложный пример: Редактор с Undo/Redo (используя Снимок и Команду)
Комбинируем паттерны Команда и Снимок для реализации полноценной отмены/повтора в текстовом редакторе.

In [2]:
import time
from abc import ABC, abstractmethod
from collections import deque

# --- 1. Memento ---
class EditorStateMemento:
    """Снимок: хранит полное состояние редактора."""
    def __init__(self, content: str, cursor_pos: int):
        # Делаем состояние неизменяемым (например, кортеж)
        self._state = (content, cursor_pos)
        self._timestamp = time.time()
        print(f"Memento: Created. Content='{self._state[0]}', Cursor={self._state[1]}")

    # 'Широкий' интерфейс для Originator
    def _get_content(self) -> str: return self._state[0]
    def _get_cursor_pos(self) -> int: return self._state[1]

    # 'Узкий' интерфейс для Caretaker
    def get_info(self) -> str:
        return f"State at {time.strftime('%H:%M:%S', time.localtime(self._timestamp))}"

# --- 2. Originator ---
class TextEditor:
    """Редактор, чьё состояние сохраняем."""
    def __init__(self):
        self._content = ""
        self._cursor_pos = 0
        print("TextEditor: Initialized")

    def insert(self, text: str):
        """Вставляет текст в позицию курсора."""
        print(f"Editor: Inserting '{text}' at {self._cursor_pos}")
        self._content = self._content[:self._cursor_pos] + text + self._content[self._cursor_pos:]
        self._cursor_pos += len(text)
        print(f"Editor: Content='{self._content}', Cursor={self._cursor_pos}")

    def delete(self, length: int):
        """Удаляет символы перед курсором."""
        if length <= 0 or self._cursor_pos == 0: return
        start = max(0, self._cursor_pos - length)
        print(f"Editor: Deleting {min(length, self._cursor_pos)} chars before {self._cursor_pos}")
        self._content = self._content[:start] + self._content[self._cursor_pos:]
        self._cursor_pos = start
        print(f"Editor: Content='{self._content}', Cursor={self._cursor_pos}")

    def move_cursor(self, offset: int):
        new_pos = max(0, min(len(self._content), self._cursor_pos + offset))
        if new_pos != self._cursor_pos:
            print(f"Editor: Moving cursor from {self._cursor_pos} to {new_pos}")
            self._cursor_pos = new_pos
        print(f"Editor: Cursor at {self._cursor_pos}")

    def get_state_info(self) -> str: # Для отображения
        return f"Content='{self._content}', Cursor={self._cursor_pos}"

    # Методы для работы со Снимком
    def save_state(self) -> EditorStateMemento:
        """Создает снимок текущего состояния."""
        print("Editor: Saving state...")
        return EditorStateMemento(self._content, self._cursor_pos)

    def restore_state(self, memento: EditorStateMemento):
        """Восстанавливает состояние из снимка."""
        self._content = memento._get_content()
        self._cursor_pos = memento._get_cursor_pos()
        print(f"Editor: State restored ({memento.get_info()}). {self.get_state_info()}")


# --- 3. Command Interface ---
#    (Используем Команду для инкапсуляции действий и управления Снимками)
class Command(ABC):
    def __init__(self, editor: TextEditor, history: 'HistoryManager'):
        self._editor = editor
        self._history = history # Команда знает об Опекуне
        self._memento_before: EditorStateMemento | None = None

    def _save_backup(self):
        self._memento_before = self._editor.save_state()

    def execute(self):
        self._save_backup() # Сохраняем состояние ДО выполнения
        self._do_execute()   # Выполняем само действие
        self._history.add_command(self) # Сообщаем Опекуну, что команда выполнена

    @abstractmethod
    def _do_execute(self): # Метод для конкретной логики команды
        pass

    def undo(self):
        if self._memento_before:
            print(f"Command ({type(self).__name__}): Undoing by restoring state...")
            self._editor.restore_state(self._memento_before)
        else:
            print(f"Command ({type(self).__name__}): Cannot undo, no previous state saved.")


# --- Concrete Commands ---
class InsertCommand(Command):
    def __init__(self, editor: TextEditor, history: 'HistoryManager', text: str):
        super().__init__(editor, history)
        self._text = text

    def _do_execute(self):
        self._editor.insert(self._text)

class DeleteCommand(Command):
    def __init__(self, editor: TextEditor, history: 'HistoryManager', length: int):
        super().__init__(editor, history)
        self._length = length

    def _do_execute(self):
        self._editor.delete(self._length)

class MoveCursorCommand(Command):
     def __init__(self, editor: TextEditor, history: 'HistoryManager', offset: int):
        super().__init__(editor, history)
        self._offset = offset

     def _do_execute(self):
        self._editor.move_cursor(self._offset)
     # Для move_cursor можно сделать undo более эффективным, запомнив старую позицию,
     # но для простоты примера используем общий механизм восстановления из Memento.


# --- 4. Caretaker (Опекун - Менеджер Истории) ---
class HistoryManager:
    """Опекун и Инициатор команд: управляет историей undo/redo."""
    def __init__(self, editor: TextEditor):
        self._editor = editor # Нужен для создания команд
        self._undo_stack = deque()
        self._redo_stack = deque()
        print("HistoryManager (Caretaker/Invoker): Initialized.")

    def execute(self, command_type: type, *args):
        """Создает, выполняет команду и управляет историей."""
        print(f"\nHistoryManager: Executing {command_type.__name__}...")
        command = command_type(self._editor, self, *args)
        command.execute()
        # При выполнении новой команды очищаем стек redo
        if self._redo_stack:
            print("HistoryManager: Clearing redo stack.")
            self._redo_stack.clear()

    # Этот метод вызывается из Command.execute()
    def add_command(self, command: Command):
        print(f"HistoryManager: Adding {type(command).__name__} to undo stack.")
        self._undo_stack.append(command)

    def undo(self):
        if not self._undo_stack:
            print("\nHistoryManager: Undo stack empty.")
            return
        command_to_undo = self._undo_stack.pop()
        print(f"\nHistoryManager: Undoing {type(command_to_undo).__name__}...")
        command_to_undo.undo()
        self._redo_stack.append(command_to_undo) # Перемещаем в стек redo
        print(f"HistoryManager: Moved {type(command_to_undo).__name__} to redo stack.")

    def redo(self):
        if not self._redo_stack:
            print("\nHistoryManager: Redo stack empty.")
            return
        command_to_redo = self._redo_stack.pop()
        print(f"\nHistoryManager: Redoing {type(command_to_redo).__name__}...")
        # Важно: перед повторным выполнением снова сохраняем состояние,
        # чтобы можно было отменить и это действие redo.
        command_to_redo.execute()
        # HistoryManager.add_command() будет вызван из execute() команды
        print(f"HistoryManager: Redo complete for {type(command_to_redo).__name__}.")


# --- 5. Client Code ---
if __name__ == "__main__":
    editor = TextEditor()
    history = HistoryManager(editor)

    history.execute(InsertCommand, "Hello ") # Добавляем "Hello "
    history.execute(InsertCommand, "World!") # Добавляем "World!"
    history.execute(MoveCursorCommand, -6)   # Перемещаем курсор перед "World!"
    history.execute(DeleteCommand, 5)       # Удаляем "Hello" (5 символов перед курсором)

    print(f"\n== Current state: {editor.get_state_info()} ==")

    history.undo() # Отменяем удаление -> Вернется "Hello World!"
    history.undo() # Отменяем перемещение курсора
    history.undo() # Отменяем вставку "World!"
    history.undo() # Отменяем вставку "Hello "

    print(f"\n== State after all undos: {editor.get_state_info()} ==")

    history.redo() # Повторяем вставку "Hello "
    history.redo() # Повторяем вставку "World!"

    print(f"\n== State after redos: {editor.get_state_info()} ==")

    # Новая команда очистит redo стек
    history.execute(InsertCommand, " Python")

    history.redo() # Пытаемся повторить - стек redo пуст

# Примерный вывод (с сокращенными сообщениями для ясности):
# TextEditor: Initialized
# HistoryManager (Caretaker/Invoker): Initialized.
#
# HistoryManager: Executing InsertCommand...
# Editor: Saving state... Memento: Created. Content='', Cursor=0
# Editor: Inserting 'Hello ' at 0 ... Content='Hello ', Cursor=6
# HistoryManager: Adding InsertCommand to undo stack.
#
# HistoryManager: Executing InsertCommand...
# Editor: Saving state... Memento: Created. Content='Hello ', Cursor=6
# Editor: Inserting 'World!' at 6 ... Content='Hello World!', Cursor=12
# HistoryManager: Adding InsertCommand to undo stack.
#
# HistoryManager: Executing MoveCursorCommand...
# Editor: Saving state... Memento: Created. Content='Hello World!', Cursor=12
# Editor: Moving cursor from 12 to 6 ... Cursor at 6
# HistoryManager: Adding MoveCursorCommand to undo stack.
#
# HistoryManager: Executing DeleteCommand...
# Editor: Saving state... Memento: Created. Content='Hello World!', Cursor=6
# Editor: Deleting 5 chars before 6 ... Content=' World!', Cursor=1
# HistoryManager: Adding DeleteCommand to undo stack.
#
# == Current state: Content=' World!', Cursor=1 ==
#
# HistoryManager: Undoing DeleteCommand...
# Command (DeleteCommand): Undoing by restoring state...
# Editor: State restored (State at <Time>). Content='Hello World!', Cursor=6
# HistoryManager: Moved DeleteCommand to redo stack.
#
# HistoryManager: Undoing MoveCursorCommand...
# Command (MoveCursorCommand): Undoing by restoring state...
# Editor: State restored (State at <Time>). Content='Hello World!', Cursor=12
# HistoryManager: Moved MoveCursorCommand to redo stack.
#
# HistoryManager: Undoing InsertCommand...
# Command (InsertCommand): Undoing by restoring state...
# Editor: State restored (State at <Time>). Content='Hello ', Cursor=6
# HistoryManager: Moved InsertCommand to redo stack.
#
# HistoryManager: Undoing InsertCommand...
# Command (InsertCommand): Undoing by restoring state...
# Editor: State restored (State at <Time>). Content='', Cursor=0
# HistoryManager: Moved InsertCommand to redo stack.
#
# == State after all undos: Content='', Cursor=0 ==
#
# HistoryManager: Redoing InsertCommand...
# Editor: Saving state... Memento: Created. Content='', Cursor=0
# Editor: Inserting 'Hello ' at 0 ... Content='Hello ', Cursor=6
# HistoryManager: Adding InsertCommand to undo stack.
# HistoryManager: Redo complete for InsertCommand.
#
# HistoryManager: Redoing InsertCommand...
# Editor: Saving state... Memento: Created. Content='Hello ', Cursor=6
# Editor: Inserting 'World!' at 6 ... Content='Hello World!', Cursor=12
# HistoryManager: Adding InsertCommand to undo stack.
# HistoryManager: Redo complete for InsertCommand.
#
# == State after redos: Content='Hello World!', Cursor=12 ==
#
# HistoryManager: Executing InsertCommand...
# Editor: Saving state... Memento: Created. Content='Hello World!', Cursor=12
# Editor: Inserting ' Python' at 12 ... Content='Hello World! Python', Cursor=19
# HistoryManager: Clearing redo stack.
# HistoryManager: Adding InsertCommand to undo stack.
#
# HistoryManager: Redo stack empty.

TextEditor: Initialized
HistoryManager (Caretaker/Invoker): Initialized.

HistoryManager: Executing InsertCommand...
Editor: Saving state...
Memento: Created. Content='', Cursor=0
Editor: Inserting 'Hello ' at 0
Editor: Content='Hello ', Cursor=6
HistoryManager: Adding InsertCommand to undo stack.

HistoryManager: Executing InsertCommand...
Editor: Saving state...
Memento: Created. Content='Hello ', Cursor=6
Editor: Inserting 'World!' at 6
Editor: Content='Hello World!', Cursor=12
HistoryManager: Adding InsertCommand to undo stack.

HistoryManager: Executing MoveCursorCommand...
Editor: Saving state...
Memento: Created. Content='Hello World!', Cursor=12
Editor: Moving cursor from 12 to 6
Editor: Cursor at 6
HistoryManager: Adding MoveCursorCommand to undo stack.

HistoryManager: Executing DeleteCommand...
Editor: Saving state...
Memento: Created. Content='Hello World!', Cursor=6
Editor: Deleting 5 chars before 6
Editor: Content='HWorld!', Cursor=1
HistoryManager: Adding DeleteCommand t